In [1]:
from __future__ import print_function
import importlib

import argparse
import torch
import torch.utils.data
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torchvision import transforms

In [2]:
%pwd

'/Users/hoangnguyen/Documents/Github/generative-model/variational-auto-encoder'

In [3]:
%cd ../datasets

/Users/hoangnguyen/Documents/Github/generative-model/datasets


In [4]:
import mnist; importlib.reload(mnist)

<module 'mnist' from '/Users/hoangnguyen/Documents/Github/generative-model/datasets/mnist.py'>

In [5]:
#Define parser
parser = {
    'batch_size' : 128,
    'epochs' : 10,
    'no_cuda' : False,
    'seed' : 2,
    'log_interval' : 10,
}

In [6]:
#use */** to pass an argument for function
# *: list
# **: dict
args = argparse.Namespace(**parser)

In [7]:
args.cuda = not args.no_cuda and torch.cuda.is_available()

In [8]:
args.cuda

False

In [9]:
torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual(args.seed)

In [10]:
%pwd

'/Users/hoangnguyen/Documents/Github/generative-model/datasets'

In [11]:
#num_workers:
#pin_memory:

kwargs = {'num_workers': 1, 'pin_memory': True} if args.cuda else {}

train_loader = torch.utils.data.DataLoader(mnist.MNIST('../data', train=True, download=True,
                transform=transforms.ToTensor()), batch_size=args.batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(mnist.MNIST('../data', train=False, transform=transforms.ToTensor()),
                batch_size=args.batch_size, shuffle=True, **kwargs)

# Define VAE

In [12]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
        #pic size 28x28 --> 784
        #400: random, 20: random
        self.fc1 = nn.Linear(784, 200) 
        self.fc21 = nn.Linear(200, 20) #encode
        self.fc22 = nn.Linear(200, 20) #encode
        self.fc3 = nn.Linear(20, 200) #decode
        self.fc4 = nn.Linear(200, 784) #decode

        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def encode(self, x):
        #x --> fc1 --> relu --> fc21
        #x --> fc1 --> relu --> fc22
        h1 = self.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)
    
# This is a method proposed by https://arxiv.org/pdf/1312.6114.pdf
    
    def reparametrize(self, mu, logvar):
        std = logvar.mul(0.5).exp_()
        if args.cuda:
            eps = torch.cuda.FloatTensor(std.size()).normal_()
        else:
            eps = torch.FloatTensor(std.size()).normal_()
        eps = Variable(eps)
        return eps.mul(std).add_(mu)

    def decode(self, z):
        #z --> fc3 --> relu --> fc4 --> sigmoid
        h3 = self.relu(self.fc3(z))
        return self.sigmoid(self.fc4(h3))

    def forward(self, x):
        #flatten input and pass to encode
        #input=batch_size x 28 x 28 --> batch_size x 784
        #mu= mean
        #logvar = log variational
        mu, logvar = self.encode(x.view(-1, 784))
        z = self.reparametrize(mu, logvar)
        return self.decode(z), mu, logvar


# Build model

In [13]:
model = VAE()

In [14]:
if args.cuda:
    model.cuda()

In [15]:
reconstruction_function = nn.BCELoss()
reconstruction_function.size_average = False

In [16]:
def loss_function(recon_x, x, mu, logvar):
    BCE = reconstruction_function(recon_x, x)

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD_element = mu.pow(2).add_(logvar.exp()).mul_(-1).add_(1).add_(logvar)
    KLD = torch.sum(KLD_element).mul_(-0.5)

    return BCE + KLD

In [17]:
#model.parameters(): update all parameters in all layers
#model.fc4(): update parameters only in fc4
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [18]:
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = Variable(data) 
        #Transform: np --> Tensor/Variable: tensor --> tensor with wrapper
        #Wraps a tensor and records the operations applied to it.
        #Variable is a thin wrapper around a Tensor object, that also holds the gradient
        if args.cuda:
            data = data.cuda()
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.data[0]
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.data[0] / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))

In [19]:
def test(epoch):
    model.eval()
    test_loss = 0
    for data, _ in test_loader:
        if args.cuda:
            data = data.cuda()
        data = Variable(data, volatile=True)
        recon_batch, mu, logvar = model(data)
        test_loss += loss_function(recon_batch, data, mu, logvar).data[0]

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [20]:
for epoch in range(1, args.epochs + 1):
    train(epoch)
    test(epoch)

Train Epoch: 1 [0/60000 (0%)]	Loss: 550.569458
Train Epoch: 1 [1280/60000 (2%)]	Loss: 374.950684
Train Epoch: 1 [2560/60000 (4%)]	Loss: 278.628448
Train Epoch: 1 [3840/60000 (6%)]	Loss: 239.884613
Train Epoch: 1 [5120/60000 (9%)]	Loss: 227.396683
Train Epoch: 1 [6400/60000 (11%)]	Loss: 216.419266
Train Epoch: 1 [7680/60000 (13%)]	Loss: 223.776917
Train Epoch: 1 [8960/60000 (15%)]	Loss: 217.389557
Train Epoch: 1 [10240/60000 (17%)]	Loss: 208.167053
Train Epoch: 1 [11520/60000 (19%)]	Loss: 206.937607
Train Epoch: 1 [12800/60000 (21%)]	Loss: 203.805969
Train Epoch: 1 [14080/60000 (23%)]	Loss: 203.577011
Train Epoch: 1 [15360/60000 (26%)]	Loss: 192.983994
Train Epoch: 1 [16640/60000 (28%)]	Loss: 185.318054
Train Epoch: 1 [17920/60000 (30%)]	Loss: 184.876953
Train Epoch: 1 [19200/60000 (32%)]	Loss: 180.848450
Train Epoch: 1 [20480/60000 (34%)]	Loss: 185.201080
Train Epoch: 1 [21760/60000 (36%)]	Loss: 179.536911
Train Epoch: 1 [23040/60000 (38%)]	Loss: 180.966690
Train Epoch: 1 [24320/60000 